In [ ]:
from fastapi import FastAPI, HTTPException, Path
from pydantic import BaseModel
import tensorflow as tf
import mysql.connector

# Definisikan model pengguna
class UserInfo(BaseModel):
    age: int
    weight: float
    height: float
    gender: int
    bmr: float
    activity_level: float

# Buat koneksi ke database MySQL
def create_db_connection():
    conn = mysql.connector.connect(
        host="23.236.58.16",
        user="root",
        password="development21",
        database="Backend_Dietin"
    )
    return conn

# Load model H5
def load_model():
    model = tf.keras.models.load_model('Model/model_1_linear.h5')
    return model

app = FastAPI()

# Endpoint untuk mendapatkan data pengguna berdasarkan ID pengguna
def get_user_info(user_id: int = Path(..., description="ID Pengguna")):
    # Buat koneksi ke database
    conn = create_db_connection()
    cursor = conn.cursor()

    # Ambil data pengguna dari database
    query = "SELECT age, weight, height, gender, bmr, activity_level FROM dataUser WHERE user_id = %s"
    cursor.execute(query, (user_id,))
    result = cursor.fetchone()

    if result is None:
        print("Pengguna dengan ID tersebut tidak ditemukan.")

    # Tutup koneksi ke database
    cursor.close()
    conn.close()

    # Mengembalikan data pengguna
    age, weight, height, gender, bmr, activity_level = result
    return {"age": age, "weight": weight, "height": height, "gender": gender, "bmr": bmr, "activity_level": activity_level}

# Endpoint untuk melakukan prediksi kalori berdasarkan data pengguna
def predict_calories(user_info: UserInfo, user_id: int = Path(..., description="ID Pengguna")):
    # Periksa apakah ada field yang kosong
    if None in user_info.dict().values():
        print("Mohon isi semua field terlebih dahulu.")

    # Buat koneksi ke database
    conn = create_db_connection()
    cursor = conn.cursor()

    # Ambil data pengguna dari database
    query = "SELECT age, weight, height, gender, bmr, activity_level FROM dataUser WHERE user_id = %s"
    cursor.execute(query, (user_id,))
    result = cursor.fetchone()

    if result is None:
        print("Pengguna dengan ID tersebut tidak ditemukan.")

    # Ambil informasi pengguna dari hasil query
    _, _, _, _, _, _ = result  # Tidak digunakan pada saat prediksi

    # Gunakan informasi pengguna untuk memprediksi kalori
    model = load_model()
    input_data = [[user_info.age, user_info.weight, user_info.height, user_info.gender, user_info.bmr, user_info.activity_level]]
    predicted_calories = model.predict(input_data)[0]

    # Simpan hasil prediksi ke dalam tabel user_info
    update_query = "UPDATE dataUser SET idealCalories = %s WHERE user_id = %s"
    cursor.execute(update_query, (predicted_calories, user_id))
    conn.commit()

    # Tutup koneksi ke database
    cursor.close()
    conn.close()

    return {"predicted_calories": predicted_calories}
